<div style="width:100%; background-color: #000041"><a target="_blank" href="bit.ly/45U3TvK"><img src="assets/YugabyteDB_DSS-Virtual_LinkedIn-Cover_1584x396.jpg" /></a></div>

<br>

> **Migrate from MySQL to Distributed, Highly Available PostgreSQL!**
>
> Registration is free  [Distributed SQL Summit](bit.ly/45U3TvK).
>

This notebook file is:

`05_Migration_Workflow_Import.ipynb`


## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Complete the steps in this notebook, *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -r MY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

---
# Migration Workflow: Import data


<div style="width:100%; background-color: #000041"><img src="assets/Migration_Workflow_04.png" /></div>

---
## Import source data
Import the source data to the target YugabyteDB database using the `yb-voyager import data`` command.

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import data  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null

You can check the status of the import.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import data status --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir 


> Did you know?
>
> You can import data from cloud storage such as:
> - AWS S3 
> - GCS buckets
> - Azure blob 
> 
> For more details, review [Import Data File](https://docs.yugabyte.com/preview/yugabyte-voyager/migrate-steps/#import-data-file) in Voyager documentation.
> 


## Import indexes and triggers
This is for Voyager 1.5. You can skip.
Import indexes and triggers to the target YugabyteDB database using the yb-voyager import schema command with an additional `--post-import-data` flag.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

# Voyager 1.5
#yb-voyager import data  \
  # --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  # --target-db-host ${DB_HOST} \
  # --target-db-port  ${DB_PORT} \
  # --target-db-name  ${DB_NAME} \
  # --target-db-user "ybvoyager" \
  # --target-db-schema "sakila" \
  # --post-import-data

## Verify migration
After the schema and data import is complete, the automated part of the database migration process is considered complete. You should manually run validation queries on both the source and target database to ensure that the data is correctly migrated. A sample query to validate the databases can include checking the row count of each table.

## Connect to YugabyteDB using the PostgreSQL Driver for Python

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql://yugabyte@'+db_host+':5433/'+db_name
print(connection_str)
engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

### Run queries

In [ ]:
%%sql
select * from sakila.actor where UPPER(first_name) = 'SCARLETT';

In [ ]:
%%sql
select count(distinct last_name) from sakila.actor;

In [ ]:
%%sql
select actor.actor_id, actor.first_name, actor.last_name,
       count(actor_id) as film_count
from sakila.actor join sakila.film_actor using (actor_id)
group by actor_id
order by film_count desc
limit 1;

### Cleanup the migration user

In [ ]:
%%sql 

REASSIGN OWNED BY ybvoyager TO yugabyte;
DROP OWNED BY ybvoyager;
DROP USER ybvoyager;

---
Delete the /tmp directory

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir

---

## 😊 Next up!
Continue your learning by opening the next notebook, `06_Migrate_to_YugabyteDB_Managed.ipynb`. 

Or, to open the notebook from GitPod, run the following:

In [ ]:
%%bash
gp open '06_Migrate_to_YugabyteDB_Managed.ipynb'